In [3]:
from apps.peoples import models as pm
from apps.activity import models as am
from apps.onboarding import models as om
from apps.y_helpdesk import models as hm
from apps.attendance import models as atm
from django.db.models.functions import Concat
from django.db.models import F, Value

